In [11]:
from config import DATABASE_URI
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import layers, models
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image
import io
import psycopg2

In [3]:
def get_images_from_db():
    
    conn = psycopg2.connect(DATABASE_URI)
    
    cursor = conn.cursor()
    cursor.execute("SELECT fruit_name, image FROM fruits")
    rows = cursor.fetchall()
    images, labels = [], []
    
    for row in rows:
        fruit_name, binary_data = row
        image = Image.open(io.BytesIO(binary_data)).convert('RGB')
        image = image.resize((128, 128))
        images.append(np.array(image))
        labels.append(fruit_name)
    cursor.close()
    conn.close()
    return np.array(images), np.array(labels)

images, labels = get_images_from_db()
print(f'Retrieved {len(images)} images from the database.')

Retrieved 9678 images from the database.


In [4]:
# Normalize the images
images = images / 255.0

# Encode labels as integers
label_encoder = LabelEncoder()

labels = label_encoder.fit_transform(labels)
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [5]:
# Build the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

c:\Users\Javier Alarcon\anaconda3\envs\dev\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     3,211,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,267,973 (12.47 MB)

 Trainable params: 3,267,973 (12.47 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 29s 116ms/step - accuracy: 0.3571 - loss: 1.4244 - val_accuracy: 0.5542 - val_loss: 1.0580
Epoch 2/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 27s 113ms/step - accuracy: 0.5636 - loss: 1.0536 - val_accuracy: 0.6064 - val_loss: 0.9833
Epoch 3/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 28s 115ms/step - accuracy: 0.6311 - loss: 0.9248 - val_accuracy: 0.6276 - val_loss: 0.9471
Epoch 4/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 28s 114ms/step - accuracy: 0.6811 - loss: 0.7893 - val_accuracy: 0.6157 - val_loss: 0.9716
Epoch 5/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 28s 115ms/step - accuracy: 0.7721 - loss: 0.6010 - val_accuracy: 0.6224 - val_loss: 1.0394
Epoch 6/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 28s 117ms/step - accuracy: 0.8678 - loss: 0.3642 - val_accuracy: 0.6265 - val_loss: 1.2198
Epoch 7/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 29s 119ms/step - accuracy: 0.9292 - loss: 0.2147 - val_accuracy: 0.6229 - val_loss: 1.5211
Epoch 8/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 28s 115ms/step - accuracy: 0.9718 - loss: 0

In [7]:
# Save the model
model.save('NeuralNetwork_train_model/fruit_classifier_model.h5')

In [21]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'Test accuracy: {test_acc}')

61/61 - 2s - 28ms/step - accuracy: 0.6209 - loss: 1.9515
Test accuracy: 0.6208677887916565


In [25]:
# Predict on test data
predictions = model.predict(X_test)
predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step


In [26]:
# Print some of the predictions to see how the model performs
for i in range(50):
    print(f'Actual: {label_encoder.inverse_transform([y_test[i]])[0]}, Predicted: {predicted_labels[i]}')

Actual: Strawberry, Predicted: Mango
Actual: Banana, Predicted: Mango
Actual: Mango, Predicted: Mango
Actual: Strawberry, Predicted: Strawberry
Actual: Apple, Predicted: Strawberry
Actual: Banana, Predicted: Banana
Actual: Mango, Predicted: Strawberry
Actual: Grape, Predicted: Strawberry
Actual: Grape, Predicted: Banana
Actual: Grape, Predicted: Strawberry
Actual: Apple, Predicted: Banana
Actual: Strawberry, Predicted: Strawberry
Actual: Apple, Predicted: Banana
Actual: Mango, Predicted: Mango
Actual: Mango, Predicted: Banana
Actual: Banana, Predicted: Banana
Actual: Mango, Predicted: Banana
Actual: Grape, Predicted: Banana
Actual: Apple, Predicted: Banana
Actual: Mango, Predicted: Strawberry
Actual: Strawberry, Predicted: Banana
Actual: Mango, Predicted: Strawberry
Actual: Strawberry, Predicted: Banana
Actual: Strawberry, Predicted: Banana
Actual: Apple, Predicted: Strawberry
Actual: Banana, Predicted: Banana
Actual: Banana, Predicted: Banana
Actual: Strawberry, Predicted: Banana
Actu

In [27]:
# Save the label encoder
import pickle

with open('label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

In [15]:
model_2 = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(label_encoder.classes_), activation='softmax')
])

model_2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model_2.summary()


c:\Users\Javier Alarcon\anaconda3\envs\dev\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_11 (Conv2D)              │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 12, 12, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 831,429 (3.17 MB)

 Trainable params: 831,429 (3.17 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
# Train the model
history_2 = model_2.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 32s 126ms/step - accuracy: 0.3941 - loss: 1.3559 - val_accuracy: 0.5795 - val_loss: 1.0161
Epoch 2/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 29s 122ms/step - accuracy: 0.5760 - loss: 1.0535 - val_accuracy: 0.6126 - val_loss: 0.9496
Epoch 3/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 29s 121ms/step - accuracy: 0.6177 - loss: 0.9538 - val_accuracy: 0.6575 - val_loss: 0.8872
Epoch 4/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 30s 124ms/step - accuracy: 0.6558 - loss: 0.8590 - val_accuracy: 0.6674 - val_loss: 0.8584
Epoch 5/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 29s 122ms/step - accuracy: 0.6781 - loss: 0.8194 - val_accuracy: 0.6467 - val_loss: 0.8874
Epoch 6/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 29s 122ms/step - accuracy: 0.7133 - loss: 0.7371 - val_accuracy: 0.6901 - val_loss: 0.7951
Epoch 7/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 30s 123ms/step - accuracy: 0.7470 - loss: 0.6520 - val_accuracy: 0.6818 - val_loss: 0.8343
Epoch 8/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 29s 121ms/step - accuracy: 0.7793 - loss: 0

In [17]:
# Save the model
model_2.save('NeuralNetwork_train_model/fruit_classifier_model_2.h5')

In [18]:
# Evaluate the model
test_loss, test_acc = model_2.evaluate(X_test, y_test, verbose=2)
print(f'Test accuracy: {test_acc}')

61/61 - 2s - 32ms/step - accuracy: 0.6705 - loss: 0.9743
Test accuracy: 0.6704545617103577


In [12]:
# Build and compile the model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(len(label_encoder.classes_), activation='softmax')(x)
model_3 = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model_3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [13]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 64, 64,    │        864 │ input_layer_3[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 64, 64,    │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 64, 64,    │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 64, 64,    │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 64, 64,    │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 64, 64,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 65, 65,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 32, 32,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 32, 32,    │      2,304 │ block_1_depthwis

 Total params: 2,340,293 (8.93 MB)

 Trainable params: 82,309 (321.52 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [14]:

# Train the model
history_3 = model_3.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))



Epoch 1/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 25s 89ms/step - accuracy: 0.7015 - loss: 0.7777 - val_accuracy: 0.8502 - val_loss: 0.4224
Epoch 2/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 20s 83ms/step - accuracy: 0.8787 - loss: 0.3409 - val_accuracy: 0.8621 - val_loss: 0.3819
Epoch 3/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 20s 83ms/step - accuracy: 0.9227 - loss: 0.2364 - val_accuracy: 0.8647 - val_loss: 0.3840
Epoch 4/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 20s 83ms/step - accuracy: 0.9461 - loss: 0.1680 - val_accuracy: 0.8419 - val_loss: 0.4670
Epoch 5/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 20s 83ms/step - accuracy: 0.9673 - loss: 0.1188 - val_accuracy: 0.8590 - val_loss: 0.4171
Epoch 6/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 20s 84ms/step - accuracy: 0.9821 - loss: 0.0732 - val_accuracy: 0.8559 - val_loss: 0.4639
Epoch 7/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 20s 82ms/step - accuracy: 0.9938 - loss: 0.0467 - val_accuracy: 0.8600 - val_loss: 0.4573
Epoch 8/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 21s 85ms/step - accuracy: 0.9984 - loss: 0.0280 - 

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'Test accuracy: {test_acc}')

In [ ]:
# Save the model
model.save('fruit_classifier_mobilenetv2.h5')